## 14 de Diciembre

In [58]:
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest, chi2_contingency, ttest_ind, mannwhitneyu


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [59]:
df = pd.read_csv("Data/raw_data_final_20231212.csv", index_col=0)

In [60]:
df.head(2)

,Age,Attrition,Businesstravel,Department,Education,Educationfield,Employeenumber,Environmentsatisfaction,Gender,Jobinvolvement,Joblevel,Jobrole,Jobsatisfaction,Maritalstatus,Monthlyrate,Numcompaniesworked,Overtime,Percentsalaryhike,Relationshipsatisfaction,Stockoptionlevel,Trainingtimeslastyear,Yearsatcompany,Yearssincelastpromotion,Yearswithcurrmanager,Datebirth,Remotework,Dailyrate,Distancefromhome,Monthlyincome,Performancerating,Totalworkingyears,Worklifebalance,Hourlyrate
0,51,No,Unknown,Research & Development,3,Unknown,162,Not Satisfied,Male,High,Executive,Research Director,Satisfied,Unknown,6462,7,No,13,Satisfied,0,5,20,15,15,1972,Yes,684.0,6.0,19537.0,3.0,13.6,3.0,51.0
1,52,No,Unknown,Unknown,4,Life Sciences,259,Satisfied,Male,Moderate,Executive,Manager,Satisfied,Unknown,5678,0,Unknown,14,Not Satisfied,1,5,33,11,9,1971,Yes,699.0,1.0,19999.0,3.0,34.0,3.0,65.0


## 1.Divide a los empleados en los grupos A y B según los criterios establecidos.

# Clasificación de los registros por grupos según la satisfacción

In [61]:
def category_satisfaction(value):
    if (value=="Satisfied") or (value=="Very Satisfied"):
        return 'A'
    else:
        return 'B'

In [62]:
df['Groups'] = df['Jobsatisfaction'].apply(category_satisfaction)

In [63]:
df["Groups"]

0       A
1       A
2       A
3       A
4       B
       ..
1609    A
1610    B
1611    A
1612    A
1613    A
Name: Groups, Length: 1510, dtype: object

# Cálculo de los porcentajes de "ATTRITION"

La métrica principal que debes usar es la tasa de rotación de empleados (Attrition) en cada grupo. Recordemos que, en esta columna, "No" indica que el empleado no ha dejado la empresa (sin rotación), mientras que "Yes" indica que el empleado ha dejado la empresa (con rotación).
Por lo tanto, los pasos que deberás seguir en esta fase son:
Divide a los empleados en los grupos A y B según los criterios establecidos.
Calcula la tasa de rotación (porcentaje de empleados que dejaron la empresa) en cada grupo.

In [64]:
group_a= df[df["Groups"]=="A"]

In [65]:
group_a.shape

(925, 34)

In [66]:
group_a["Attrition"].value_counts()/group_a.shape[0]*100

Attrition
No     86.27027
Yes    13.72973
Name: count, dtype: float64

In [67]:
group_b= df[df["Groups"]=="B"]

In [68]:
group_b.shape

(585, 34)

In [69]:
group_b["Attrition"].value_counts()/group_b.shape[0]*100

Attrition
No     80.512821
Yes    19.487179
Name: count, dtype: float64

In [70]:
rotacion_por_grupo = df[df['Attrition'] == 'Yes'].groupby('Groups').size() / df.groupby('Groups').size() * 100

In [71]:
# Calcular la tasa de rotación por grupo en porcentaje
rotacion_por_grupo = df[df['Attrition'] == 'Yes'].groupby('Groups').size() / df.groupby('Groups').size() * 100

# Imprimir los resultados
print("La tasa de rotación de cada grupo es:")
for grupo, tasa in rotacion_por_grupo.items():
    print(f"Grupo {grupo}: {tasa}")


La tasa de rotación de cada grupo es:
Grupo A: 13.729729729729732
Grupo B: 19.48717948717949


# Prueba Estadística

Comprobar los tres supuestos para pruebas paramétricas: 
    Evalúa la normalidad de una columna de datos de un DataFrame utilizando la prueba de Shapiro-Wilk.

    Parámetros:
        dataframe (DataFrame): El DataFrame que contiene los datos.
        columna (str): El nombre de la columna en el DataFrame que se va a evaluar para la normalidad.

    Returns:
        None: Imprime un mensaje indicando si los datos siguen o no una distribución normal.
    """

    statistic, p_value = stats.shapiro(dataframe[columna])
    if p_value > 0.05:
        print(f"Para la columna {columna} los datos siguen una distribución normal.")
    else:
        print(f"Para la columna {columna} los datos no siguen una distribución normal.")

# 15 de Diciembre

Antes de hacer la prueba estadística:

 - para cálculos donde se necesita valor numérico, vamos a hacer un mapeo de los valores de Attrition, ya que es de tipo object. Para ello creamos este mapeo (Yes = 1 y No = 0)
 
 - para los cálculos donde se permiten valores tipo object (chi-cuadrado), utilizamos la columna Attrition original

In [72]:
# utilizamos esta columna de nueva creación en los cálculos de estadísticas que necesitan valores numéricos
df['Attrition_num'] = df['Attrition'].map({'Yes':1, 'No':0})

### Análisis de normalidad

In [73]:
# Análisis de normalidad:

# Función para testar la normalidad de los datos
def normalidad(dataframe, columna):
    statistic, p_value = stats.shapiro(dataframe[columna])
    if p_value > 0.05:
        print(f"Para la columna {columna} los datos siguen una distribución normal.")
    else:
        print(f"Para la columna {columna} los datos no siguen una distribución normal.")

In [74]:
metricas = ['Attrition_num']

for metrica in metricas:
    normalidad(df ,metrica)

Para la columna Attrition_num los datos no siguen una distribución normal.


### Análisis de la homogeneidad de varianzas

In [75]:
# Creamos función para evaluar la homogeneidad de las varianzas entre grupos para la métrica Attrition_num de nuestro df

def homogeneidad (dataframe, columna, columna_metrica):
     
    # lo primero que tenemos que hacer es crear tantos conjuntos de datos para cada una de las categorías que tenemos, Control Campaign y Test Campaign
    valores_evaluar = []
    
    for valor in dataframe[columna].unique():
        valores_evaluar.append(dataframe[dataframe[columna]== valor][columna_metrica])

    statistic, p_value = stats.levene(*valores_evaluar)
    if p_value > 0.05:
        print(f"Para la métrica {columna_metrica} las varianzas son homogéneas entre grupos.")
    else:
        print(f"Para la métrica {columna_metrica}, las varianzas no son homogéneas entre grupos.")

In [76]:
metricas = ['Attrition_num']

for metrica in metricas:
    homogeneidad(df ,'Groups', metrica)

Para la métrica Attrition_num, las varianzas no son homogéneas entre grupos.


### Análisis de la Independencia de las observaciones

In [77]:
# aquí volvemos a utilizar Attrition
# Realizar la prueba de chi-cuadrado para evaluar la independencia de dos variables categóricas
contingency_table = pd.crosstab(df['Groups'], df['Attrition'])
chi2, p_chi2, _, _ = chi2_contingency(contingency_table)
print(f"Prueba de Chi-cuadrado - p-valor: {p_chi2}")

Prueba de Chi-cuadrado - p-valor: 0.0036860670797145547


In [78]:
# para este análisis, necesitamos dato numérico, utilizamos Attrition_num
# Realizar una prueba t para comparar las medias de dos grupos (por ejemplo, Control vs. Test)
control_group = df[df['Groups'] == 'A']
test_group = df[df['Groups'] == 'B']
t_stat, p_ttest = ttest_ind(control_group['Attrition_num'], test_group['Attrition_num'])
print(f"Prueba t para Attrition entre Grupo A y Grupo B y Test - p-valor: {p_ttest}")

Prueba t para Attrition entre Grupo A y Grupo B y Test - p-valor: 0.0029025283865571736


Con todo lo anterior se concluye que <span style="color:#D4AF37"> nuestros datos no cumplen el supuesto de normalidad ni el de homogeneidad</span>.

Vamos a realizar pruebas no paramétricas, concretamente el **Test de chi-cuadrado**.

In [82]:
#Creamos una función para el cálculo del Test de chi-cuadrado
def statistics_test(column1, column2):

    # Crear una tabla de contingencia
    contingency_table = pd.crosstab(column1, column2)

    # Ejecutar el test de chi-cuadrado
    chi2, p_valor, _, _ = chi2_contingency(contingency_table)

    # Mostrar los resultados del test
    print("\nResultados del test de Chi-cuadrado:")
    print(f"Estadística de Chi-cuadrado: {chi2}")
    print(f"P-valor: {p_valor}")

    # Interpretación del resultado
    alpha = 0.05  # Nivel de significancia
    if p_valor < alpha:
        print("\nPara la métrica de Attrition existe evidencia suficiente para rechazar la hipótesis nula.")
    else:
        print("\nNo hay suficiente evidencia para rechazar la hipótesis nula.")

In [83]:
statistics_test(df['Attrition'], df['Groups'])


Resultados del test de Chi-cuadrado:
Estadística de Chi-cuadrado: 8.432334983446875
P-valor: 0.0036860670797145547

Para la métrica de Attrition existe evidencia suficiente para rechazar la hipótesis nula.


### Interpretación de los resultados

- Los datos sugieren que existe una diferencia significativa en la tasa de rotación entre los grupos de mayor y menor satisfacción laboral, ya que los p-valores son menores que 0.05.

> **Conclusión**: Resultado del análisis estadístico de los datos sugiere que <span style="color:#D4AF37"> existen diferencias significativas entre la tasa de rotación de empleados con mayor y menor satisfacción laboral</span>.

### 4. Calculamos la magnitud de la relación entre cada grupo A-B y su Attrition.

-  "Diferencia de medias y medianas"

In [80]:
# Calcular la diferencia entre las medias
media_grupo_A = df[df['Groups'] == 'A']['Attrition_num'].mean()
media_grupo_B = df[df['Groups'] == 'B']['Attrition_num'].mean()
diferencia_medias = media_grupo_B - media_grupo_A

# Calcular la diferencia entre las medianas
mediana_grupo_A = df[df['Groups'] == 'A']['Attrition_num'].median()
mediana_grupo_B = df[df['Groups'] == 'B']['Attrition_num'].median()
diferencia_medianas = mediana_grupo_B - mediana_grupo_A

print("Diferencia de medias:", diferencia_medias)
print("Diferencia de medianas:", diferencia_medianas)


Diferencia de medias: 0.05757449757449756
Diferencia de medianas: 0.0


> **Conclusión**: 

> El análisis diferencial de medianas nos sugiere que 

> El análisis diferencial de medias sugiere que, en promedio, <span style="color:#D4AF37">la tasa de rotación es ligeramente más alta en el Grupo B *pobre satisfacción laboral* en comparación con el Grupo A *alta satisfacción laboral*</span>, con una magnitud de *aproximadamente 0.057575*.

 - Prueba de Mann-Whitney U:

In [81]:
# Prueba de Mann-Whitney U
stat, p_valor = mannwhitneyu(df[df['Groups'] == 'A']['Attrition_num'], df[df['Groups'] == 'B']['Attrition_num'])
tamaño_efecto = abs((stat - (len(df[df['Groups'] == 'A']['Attrition_num']) * len(df[df['Groups'] == 'B']['Attrition_num']) / 2)) / np.sqrt(len(df[df['Groups'] == 'A']['Attrition_num']) * len(df[df['Groups'] == 'B']['Attrition_num'])))
print("Prueba de Mann-Whitney U - p-valor:", p_valor)
print("Tamaño de efecto:", tamaño_efecto)

# Evaluar el p-valor
nivel_de_significancia = 0.05  # Nivel de significancia comúnmente usado

if p_valor < nivel_de_significancia:
    print("Se rechaza la hipótesis nula.")
    print("Hay evidencia suficiente para sugerir que existe una diferencia significativa entre los grupos.")
else:
    print("No se puede rechazar la hipótesis nula.")
    print("No hay suficiente evidencia para afirmar que existe una diferencia significativa entre los grupos.")

Prueba de Mann-Whitney U - p-valor: 0.0029309760646217035
Tamaño de efecto: 21.176245370305093
Se rechaza la hipótesis nula.
Hay evidencia suficiente para sugerir que existe una diferencia significativa entre los grupos.


> **Conclusión**:
> El **p-valor de 0.0029** sugiere que hay evidencia estadística significativa para rechazar la hipótesis nula. Esto sugiere que <span style="color:#D4AF37"> existe una diferencia significativa entre las distribuciones de las tasas de rotación entre los dos grupos.</span>. La magnitud de esa diferencia es grande, tal y como sugiere el **Tamaño de efecto 21.176** , lo que sugiere un <span style="color:#D4AF37"> la magnitud de la diferencia entre los el grupo A (alta satisfacción laboral) y el grupo B (baja satisfacción laboral) en términos de la magnitud de la diferencia en las medianas de las distribuciones de los grupos</span>.